<a href="https://colab.research.google.com/github/quanndm/kltn/blob/dev/notebooks/tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Init - import library - download data

In [1]:
!rm -rf kltn
!rm -rf sample_data
!git clone -b dev https://github.com/quanndm/kltn.git

Cloning into 'kltn'...
remote: Enumerating objects: 1248, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 1248 (delta 19), reused 21 (delta 8), pack-reused 1212 (from 2)
Receiving objects: 100% (1248/1248), 13.28 MiB | 30.92 MiB/s, done.
Resolving deltas: 100% (807/807), done.


In [2]:
from kltn.init.install_dependencies import install_packages, load_config
install_packages("./kltn")
config = load_config("./kltn/parameters.yaml")

In [3]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive

from kltn import *
from kltn.datasets.get_datasets import get_datasets_stage2
from kltn.utils.logger import init_logger
from kltn.utils.train_val_epochs import trainer, trainer_stage2
from kltn.utils.metrics import  DiceMetric, TverskyLossWSigmoid
from kltn.models.model_factory import ModelFactory
from kltn.utils.visualize_results import visualize_results

In [4]:
drive.flush_and_unmount()
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
LOGGER = init_logger(config["log_path"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
from kltn.datasets.prepare_data import prepare_dataset_LiTS
prepare_dataset_LiTS(dir_name=config["source_folder_lits"])

0.00% complete (down: 0.0 kB/s up: 0.0 kB/s peers: 2) downloading LITS17 (https://academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (https://academictorrents.com/announce.php)[172.28.0.12:6881] v1 unspecified system error "" (1)
LITS17 (https://ipv6.academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (udp://tracker.opentrackr.org:1337/announce)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (1)
LITS17 (https://ipv6.academictorrents.com/announce.php)[172.28.0.12:6881] v1 unspecified system error "" (1)
1.79% complete (down: 25648.9 kB/s up: 740.1 kB/s peers: 10) downloading LITS17 (https://academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (2)
LITS17 (https://ipv6.academictorrents.com/announce.php)[127.0.0.1:6881] v1 skipping tracker announce (unreachable) "" (2)
LITS17 (https://academictorrents.com/announce.php)[1

## Model

In [6]:
weight_path = "/content/drive/MyDrive/KLTN/code/weight/best_metric_model_UNet3DWResNeXtCoT.pth"
model_stage_1 = ModelFactory.get_model("unet3d_resnextcot",in_channels=1, n_classes=3, n_channels=16).to(device)
model_stage_1.load_state_dict(torch.load(weight_path, map_location=device))

<All keys matched successfully>

In [7]:
model = ModelFactory.get_model("unet3d_resnextcot",in_channels=1, n_classes=1, n_channels=16).to(device)
model.load_state_dict({
    k: v for k, v in model_stage_1.state_dict().items() if "out" not in k
}, strict=False)

_IncompatibleKeys(missing_keys=['out.conv.weight', 'out.conv.bias'], unexpected_keys=[])

## Load data

In [8]:
full_train_dataset, val_dataset = get_datasets_stage2(source_folder=config["source_folder_lits"], seed=123, fold_number=2, model_stage_1=model_stage_1)
print(len(full_train_dataset), len(val_dataset))

105 26


In [9]:
train_loader = torch.utils.data.DataLoader(full_train_dataset, batch_size=1, shuffle=True,
                                           num_workers=0, pin_memory=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False,
                                         pin_memory=True, num_workers=0)

## Training

In [10]:
criterion = TverskyLossWSigmoid().to(device)

dice_acc = DiceMetric(include_background=False, reduction='mean_batch', get_not_nans=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=float(config["learning_rate_train"]), weight_decay=float(config["weight_decay"]))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=int(config["max_epochs"]), eta_min = float(config["eta_min"]))

In [ ]:
LOGGER.info("[TRAINER] Start TRAIN process...")

(
    val_acc_max,
    best_epoch,
    dices_tumor,
    loss_epochs,
    trains_epoch,
    ious_tumor,
    precisions_tumor,
    recalls_tumor,
    time_tmp
) = trainer_stage2(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    loss_func=criterion,
    acc_func=dice_acc,
    scheduler=scheduler,
    batch_size=config["batch_size"],
    max_epochs = 10,
    start_epoch = config["start_epoch"],
    val_every=config["val_every"],
    path_save_model=config["path_save_model_state"],
    logger=LOGGER,
    save_model=False,
    post_fix="tumor"
)

[TRAINER] Start TRAIN process...
INFO:kltn.utils.logger:[TRAINER] Start TRAIN process...

==============================Training epoch 1==============================
INFO:kltn.utils.logger:
==============================Training epoch 1==============================


Epoch 1/10 1/105 loss: 0.8823 time 4.45s
Epoch 1/10 2/105 loss: 0.8903 time 5.66s
Epoch 1/10 3/105 loss: 0.8845 time 1.26s
Epoch 1/10 4/105 loss: 0.8624 time 4.60s
Epoch 1/10 5/105 loss: 0.8631 time 4.61s
Epoch 1/10 6/105 loss: 0.8407 time 5.82s
Epoch 1/10 7/105 loss: 0.8446 time 1.22s
Epoch 1/10 8/105 loss: 0.8399 time 1.29s
Epoch 1/10 9/105 loss: 0.8417 time 2.34s
Epoch 1/10 10/105 loss: 0.8420 time 2.69s
Epoch 1/10 11/105 loss: 0.8437 time 2.22s
Epoch 1/10 12/105 loss: 0.8466 time 4.52s
Epoch 1/10 13/105 loss: 0.8481 time 4.69s
Epoch 1/10 14/105 loss: 0.8510 time 4.23s
Epoch 1/10 15/105 loss: 0.8507 time 1.69s
Epoch 1/10 16/105 loss: 0.8417 time 5.79s
Epoch 1/10 17/105 loss: 0.8357 time 1.35s
Epoch 1/10 18/105 loss: 0.8366 time 5.61s
Epoch 1/10 19/105 loss: 0.8376 time 6.57s
Epoch 1/10 20/105 loss: 0.8376 time 4.98s
Epoch 1/10 21/105 loss: 0.8364 time 4.98s
Epoch 1/10 22/105 loss: 0.8369 time 4.49s
Epoch 1/10 23/105 loss: 0.8348 time 1.78s
Epoch 1/10 24/105 loss: 0.8334 time 1.29s
E

Final training epochs: 1/10 ---[loss: 0.7961] ---[time 415.88s]
INFO:kltn.utils.logger:Final training epochs: 1/10 ---[loss: 0.7961] ---[time 415.88s]

********************Epoch 1 Validation********************
INFO:kltn.utils.logger:
********************Epoch 1 Validation********************


Epoch 1/10 105/105 loss: 0.7961 time 7.45s


Val 1/10 1/26, Dice_Tumor: 0.000000, time 2.15s
INFO:kltn.utils.logger:Val 1/10 1/26, Dice_Tumor: 0.000000, time 2.15s
Val 1/10 2/26, Dice_Tumor: 0.000000, time 2.95s
INFO:kltn.utils.logger:Val 1/10 2/26, Dice_Tumor: 0.000000, time 2.95s
Val 1/10 3/26, Dice_Tumor: 0.000000, time 0.96s
INFO:kltn.utils.logger:Val 1/10 3/26, Dice_Tumor: 0.000000, time 0.96s
Val 1/10 4/26, Dice_Tumor: 0.011399, time 3.03s
INFO:kltn.utils.logger:Val 1/10 4/26, Dice_Tumor: 0.011399, time 3.03s
Val 1/10 5/26, Dice_Tumor: 0.000000, time 6.00s
INFO:kltn.utils.logger:Val 1/10 5/26, Dice_Tumor: 0.000000, time 6.00s
Val 1/10 6/26, Dice_Tumor: 0.013225, time 4.09s
INFO:kltn.utils.logger:Val 1/10 6/26, Dice_Tumor: 0.013225, time 4.09s
Val 1/10 7/26, Dice_Tumor: 0.001871, time 1.92s
INFO:kltn.utils.logger:Val 1/10 7/26, Dice_Tumor: 0.001871, time 1.92s
Val 1/10 8/26, Dice_Tumor: 0.008671, time 1.65s
INFO:kltn.utils.logger:Val 1/10 8/26, Dice_Tumor: 0.008671, time 1.65s
Val 1/10 9/26, Dice_Tumor: 0.000000, time 5.66s


...

Val 5/10 1/26, Dice_Tumor: 0.167875, time 2.16s
INFO:kltn.utils.logger:Val 5/10 1/26, Dice_Tumor: 0.167875, time 2.16s
Val 5/10 2/26, Dice_Tumor: 0.001418, time 3.14s
INFO:kltn.utils.logger:Val 5/10 2/26, Dice_Tumor: 0.001418, time 3.14s
Val 5/10 3/26, Dice_Tumor: 0.000000, time 1.01s
INFO:kltn.utils.logger:Val 5/10 3/26, Dice_Tumor: 0.000000, time 1.01s
Val 5/10 4/26, Dice_Tumor: 0.543585, time 3.29s
INFO:kltn.utils.logger:Val 5/10 4/26, Dice_Tumor: 0.543585, time 3.29s
Val 5/10 5/26, Dice_Tumor: 0.000000, time 6.57s
INFO:kltn.utils.logger:Val 5/10 5/26, Dice_Tumor: 0.000000, time 6.57s
Val 5/10 6/26, Dice_Tumor: 0.026709, time 4.42s
INFO:kltn.utils.logger:Val 5/10 6/26, Dice_Tumor: 0.026709, time 4.42s
Val 5/10 7/26, Dice_Tumor: 0.199580, time 2.12s
INFO:kltn.utils.logger:Val 5/10 7/26, Dice_Tumor: 0.199580, time 2.12s
Val 5/10 8/26, Dice_Tumor: 0.557964, time 1.62s
INFO:kltn.utils.logger:Val 5/10 8/26, Dice_Tumor: 0.557964, time 1.62s
Val 5/10 9/26, Dice_Tumor: 0.000000, time 6.08s


New best (0.501373 --> 0.533271). At epoch 5
Epoch 6/10 1/105 loss: 0.7281 time 7.25s
Epoch 6/10 2/105 loss: 0.6382 time 1.38s
Epoch 6/10 3/105 loss: 0.6687 time 2.79s
Epoch 6/10 4/105 loss: 0.6136 time 5.11s
Epoch 6/10 5/105 loss: 0.6231 time 6.65s
Epoch 6/10 6/105 loss: 0.6396 time 4.58s
Epoch 6/10 7/105 loss: 0.6486 time 1.13s
Epoch 6/10 8/105 loss: 0.6593 time 2.96s
Epoch 6/10 9/105 loss: 0.6463 time 6.27s
Epoch 6/10 10/105 loss: 0.6528 time 4.61s
Epoch 6/10 11/105 loss: 0.6530 time 2.56s
Epoch 6/10 12/105 loss: 0.6558 time 7.11s
Epoch 6/10 13/105 loss: 0.6575 time 1.29s
Epoch 6/10 14/105 loss: 0.6617 time 4.68s
Epoch 6/10 15/105 loss: 0.6658 time 1.85s
Epoch 6/10 16/105 loss: 0.6690 time 2.00s
Epoch 6/10 17/105 loss: 0.6695 time 1.95s
Epoch 6/10 18/105 loss: 0.6725 time 5.66s
Epoch 6/10 19/105 loss: 0.6684 time 5.78s
Epoch 6/10 20/105 loss: 0.6697 time 1.88s
Epoch 6/10 21/105 loss: 0.6720 time 6.18s
Epoch 6/10 22/105 loss: 0.6732 time 5.61s
Epoch 6/10 23/105 loss: 0.6680 time 7.36

Final training epochs: 6/10 ---[loss: 0.6454] ---[time 445.45s]
INFO:kltn.utils.logger:Final training epochs: 6/10 ---[loss: 0.6454] ---[time 445.45s]

********************Epoch 6 Validation********************
INFO:kltn.utils.logger:
********************Epoch 6 Validation********************


Epoch 6/10 105/105 loss: 0.6454 time 5.17s


Val 6/10 1/26, Dice_Tumor: 0.195748, time 1.74s
INFO:kltn.utils.logger:Val 6/10 1/26, Dice_Tumor: 0.195748, time 1.74s
Val 6/10 2/26, Dice_Tumor: 0.001114, time 3.65s
INFO:kltn.utils.logger:Val 6/10 2/26, Dice_Tumor: 0.001114, time 3.65s
Val 6/10 3/26, Dice_Tumor: 0.088898, time 1.20s
INFO:kltn.utils.logger:Val 6/10 3/26, Dice_Tumor: 0.088898, time 1.20s
Val 6/10 4/26, Dice_Tumor: 0.577105, time 3.25s
INFO:kltn.utils.logger:Val 6/10 4/26, Dice_Tumor: 0.577105, time 3.25s
Val 6/10 5/26, Dice_Tumor: 0.000000, time 6.36s
INFO:kltn.utils.logger:Val 6/10 5/26, Dice_Tumor: 0.000000, time 6.36s
Val 6/10 6/26, Dice_Tumor: 0.034857, time 4.73s
INFO:kltn.utils.logger:Val 6/10 6/26, Dice_Tumor: 0.034857, time 4.73s
Val 6/10 7/26, Dice_Tumor: 0.232781, time 2.20s
INFO:kltn.utils.logger:Val 6/10 7/26, Dice_Tumor: 0.232781, time 2.20s
Val 6/10 8/26, Dice_Tumor: 0.572848, time 1.63s
INFO:kltn.utils.logger:Val 6/10 8/26, Dice_Tumor: 0.572848, time 1.63s
Val 6/10 9/26, Dice_Tumor: 0.000000, time 6.74s


In [ ]:
import json
path_save_result = f"/content/drive/MyDrive/KLTN/code/result_model_{model.__class__.__name__}_tumor.json"

results = {
    "dice_tumor": [float(x) for x in np.array(dices_tumor, dtype=np.float32)],
    "loss": [float(x) for x in np.array(loss_epochs, dtype= np.float32)],
    "iou_tumor": [float(x) for x in np.array(ious_tumor, dtype=np.float32)],
    "precision_tumor": [float(x) for x in np.array(precisions_tumor, dtype=np.float32)],
    "recall_tumor": [float(x) for x in np.array(recalls_tumor, dtype=np.float32)],
    "best_epoch": best_epoch,
    "time_train": time_tmp
}
with open(path_save_result, "w") as f:
    json.dump(results, f, indent=4)